### Python Automation and Parallel Processing

#### Import Liabrires

In [5]:
import pandas as pd
import numpy as mp
import os, re, requests, time
import logging as log
from datetime import date, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from pandas.errors import EmptyDataError, ParserError
log.basicConfig(level=log.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Market Data Module

In [14]:
tradeData = pd.read_csv('Scripts.csv',index_col='CUSIP')
tradeData

,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price per $100,Accrued Interest per $100,"Accrued Interest per $1,000","Adjusted Accrued Interest per $1,000",Adjusted Price,...,"TIIN Conversion Factor per $1,000",TIPS,Total Accepted,Total Tendered,Treasury Retail Accepted,Treasury Retail Tenders Accepted,"Unadjusted Accrued Interest per $1,000",Unadjusted Price,XML Filename - Announcement,XML Filename - Competitive Results
CUSIP,,,,,,,,,,,,,,,,,,,,,
91282CNQ0,Note,2-Year,07/29/2025,07/31/2025,07/31/2027,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,Yes,NaN,NaN,A_20250724_6.xml,NaN
91282CNR8,Note,7-Year,07/29/2025,07/31/2025,07/31/2032,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,Yes,NaN,NaN,A_20250724_4.xml,NaN
91282CNP2,Note,2-Year,07/28/2025,07/31/2025,07/31/2027,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,Yes,NaN,NaN,A_20250724_3.xml,NaN
91282CNN7,Note,5-Year,07/28/2025,07/31/2025,07/31/2030,NaN,NaN,NaN,NaN,NaN,...,NaN,No,NaN,NaN,NaN,Yes,NaN,NaN,A_20250724_1.xml,NaN
91282CNS6,Note,10-Year,07/24/2025,07/31/2025,07/15/2035,99.116833,NaN,NaN,$0.81610,99.116833,...,2.919673,Yes,"$22,430,027,400","$52,135,717,500","$14,284,900",Yes,$0.81522,99.009902,A_20250717_4.xml,R_20250724_3.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91282CBE0,Note,3-Year,01/11/21,01/15/2021,01/15/2024,99.674335,NaN,NaN,NaN,NaN,...,NaN,No,"$63,830,661,800","$152,016,891,800","$10,836,800",Yes,NaN,NaN,A_20210107_5.xml,R_20210111_3.xml
91282CBB6,Note,7-Year,12/29/2020,12/31/2020,12/31/2027,99.747318,NaN,NaN,NaN,NaN,...,NaN,No,"$67,976,515,100","$145,552,785,700","$2,072,700",Yes,NaN,NaN,A_20201224_1.xml,R_20201229_4.xml
91282CBD2,Note,2-Year,12/28/2020,12/31/2020,12/31/2022,99.976041,NaN,NaN,NaN,NaN,...,NaN,No,"$66,824,414,100","$151,105,026,600","$81,119,800",Yes,NaN,NaN,A_20201224_6.xml,R_20201228_2.xml


In [12]:
os.cpu_count()

8

In [34]:
class MarketDataExtracts:

    def __init__(self, tradeData, path=None):
        self.tradeData, self.path = tradeData, path
        self._ValidateInputs()
        
    def _ValidateInputs(self):
        
        if self.tradeData is None or self.tradeData.empty:
            raise ValueError(f"Trade Data Not Found")

        if not os.path.exists(self.path):
            raise ValueError(f"Path Doesn't Exists:{self.path}")

        if not os.path.isdir(self.path):
            raise ValueError(f"Path Not A Directory:{self.path}")

        if not os.access(self.path, os.W_OK):
            raise ValueError(f"Path is Not Writable:{self.path}")

    def ProcessAuctionResults(self, parallel=False, threads=1):

        startTime = time.time()
        log.info(f'Processing Results for{len(self.tradeData.index)} CUSIPs - Mode: {'Parallel'if parallel else "Sequential"}')

        try:
            if parallel:
                self.ProcessResultsInParallel(threads)
            
            if not parallel:
                self.ProcessResultsInSequnce()
        
        except Exception as e:
            log.info(f'Error Encountered Processing results:{str(e)}')

        processingTime = time.time() - startTime
        log.info(f'Proccessed{len(self.tradeData.index)} Objects in {processingTime:.2f} Seconds')

    def ProcessResultsInSequnce(self):

        _results, f_results=[], []
        for cusip in self.tradeData.index:
            status = self._ProcessResult(cusip)
            if status:
                _results.append(cusip)
            if not status:
                f_results.append(cusip)
        
    def ProcessResultsInParallel(self, threads):
        
        _results, f_results=[], []
        with ThreadPoolExecutor(max_workers=threads) as executor:
            futures = {executor.submit(self._ProcessResult, cusip): cusip for cusip in self.tradeData.index}
            for future in as_completed(futures):
                status = future.result()
                if status:
                    _results.append(cusip)
                if not status:
                    f_results.append(cusip)
    
    
    def _ProcessResult(self, cusip):

        try:
            Year = pd.to_datetime(self.tradeData.loc[cusip,"Auction Date"]).year
            pdfName = self.tradeData.loc[cusip,"PDF Filename - Competitive Results"]
            url = f"https://www.treasurydirect.gov/instit/annceresult/press/preanre/{Year}/{pdfName}"
            pathPDF = f"{self.path}/{pdfName}"
            pathPDF_ = f'{self.path}/Result_{cusip}.pdf'
    
            response = requests.get(url,timeout=30)
    
            if response.status_code == 200:
                with open(pathPDF,"wb") as _path:
                    _path.write(response.content)
                os.rename(pathPDF,pathPDF_)
                log.info(f'Processed CUSIP:{cusip}')
                return True
            
            else:
                log.error(f'HTTP Request{response.status_code} for CUSIP {cusip}')
                return False

        except Exception as e:
            log.error(f'Exception Processing CUSIP {cusip}: {str(e)}')
            return False

In [20]:
dataFunction = MarketDataExtracts(tradeData,path='PDF')
dataFunction.ProcessAuctionResults(parallel=False, threads=1)

2025-07-27 15:28:03,282 - root - INFO - Processing Results for321 CUSIPs - Mode: Sequential
2025-07-27 15:28:04,807 - root - ERROR - HTTP Request404 for CUSIP 91282CNQ0
2025-07-27 15:28:05,981 - root - ERROR - HTTP Request404 for CUSIP 91282CNR8
2025-07-27 15:28:07,419 - root - ERROR - HTTP Request404 for CUSIP 91282CNP2
2025-07-27 15:28:08,566 - root - ERROR - HTTP Request404 for CUSIP 91282CNN7
2025-07-27 15:28:10,402 - root - INFO - Processed CUSIP:91282CNS6
2025-07-27 15:28:12,129 - root - INFO - Processed CUSIP:91282CNM9
2025-07-27 15:28:13,873 - root - INFO - Processed CUSIP:91282CNJ6
2025-07-27 15:28:15,606 - root - INFO - Processed CUSIP:91282CNK3
2025-07-27 15:28:17,357 - root - INFO - Processed CUSIP:91282CNL1
2025-07-27 15:28:19,136 - root - INFO - Processed CUSIP:91282CNH0
2025-07-27 15:28:20,872 - root - INFO - Processed CUSIP:91282CNF4
2025-07-27 15:28:22,695 - root - INFO - Processed CUSIP:91282CNG2
2025-07-27 15:28:24,194 - root - INFO - Processed CUSIP:91282CNE7
2025-0

In [36]:
dataFunction = MarketDataExtracts(tradeData,path='PDF')
dataFunction.ProcessAuctionResults(parallel=True, threads=os.cpu_count()-1)

2025-07-27 15:50:49,459 - root - INFO - Processing Results for321 CUSIPs - Mode: Parallel
2025-07-27 15:50:51,046 - root - ERROR - HTTP Request404 for CUSIP 91282CNR8
2025-07-27 15:50:51,052 - root - ERROR - HTTP Request404 for CUSIP 91282CNQ0
2025-07-27 15:50:51,100 - root - ERROR - HTTP Request404 for CUSIP 91282CNP2
2025-07-27 15:50:51,157 - root - ERROR - HTTP Request404 for CUSIP 91282CNN7
2025-07-27 15:50:51,357 - root - INFO - Processed CUSIP:91282CNJ6
2025-07-27 15:50:51,360 - root - INFO - Processed CUSIP:91282CNS6
2025-07-27 15:50:51,388 - root - INFO - Processed CUSIP:91282CNM9
2025-07-27 15:50:52,618 - root - INFO - Processed CUSIP:91282CNL1
2025-07-27 15:50:52,856 - root - INFO - Processed CUSIP:91282CNK3
2025-07-27 15:50:52,860 - root - INFO - Processed CUSIP:91282CNF4
2025-07-27 15:50:52,936 - root - INFO - Processed CUSIP:91282CNH0
2025-07-27 15:50:53,162 - root - INFO - Processed CUSIP:91282CNG2
2025-07-27 15:50:53,229 - root - INFO - Processed CUSIP:91282CNE7
2025-07-